Starting this attempt small with just looking for Belgium 2023 Grand Prix

In [1]:
import time
import random
from urllib.request import urlopen
import json
import pandas as pd

def get_with_backoff(url, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            response = urlopen(url)
            return response
        except HTTPError as e:
            if e.code == 429:
                wait_time = (2 ** retries) + random.random()
                print(f"Rate limited. Waiting {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1
            else:
                raise
    raise Exception("Max retries exceeded")

## Meeting data

In [ ]:
# specifically looking for Belgium 2023
response = urlopen('https://api.openf1.org/v1/meetings?year=2023&country_name=Belgium')
data = json.loads(response.read().decode('utf-8'))
meeting = pd.DataFrame(data)
meeting.head(10)

# meeting_key = 1216

,meeting_name,meeting_official_name,location,country_key,country_code,country_name,circuit_key,circuit_short_name,date_start,gmt_offset,meeting_key,year
0,Belgian Grand Prix,FORMULA 1 MSC CRUISES BELGIAN GRAND PRIX 2023,Spa-Francorchamps,16,BEL,Belgium,7,Spa-Francorchamps,2023-07-28T11:30:00+00:00,02:00:00,1216,2023


## Session data

In [3]:
# to find the final race data
response = urlopen('https://api.openf1.org/v1/sessions?meeting_key=1216')
data = json.loads(response.read().decode('utf-8'))
df = pd.DataFrame(data)

In [ ]:
df[['session_type', 'session_name', 'session_key', 'meeting_key']].head(10)

,session_type,session_name,session_key,meeting_key
0,Practice,Practice 1,9134,1216
1,Qualifying,Qualifying,9135,1216
2,Qualifying,Sprint Shootout,9286,1216
3,Race,Sprint,9140,1216
4,Race,Race,9141,1216


So we see here the session keys corresponding to the diff sessions for this grand prix. we'll use practice and two qualifying for our training data i guess? 

## Position data

In [2]:
response = urlopen('https://api.openf1.org/v1/position')
data = json.loads(response.read().decode('utf-8'))
df = pd.DataFrame(data)

HTTPError: HTTP Error 500: Internal Server Error